# Tema 1 IA - Class Scheduler
### Alexandru LICURICEANU - 332CD

In [223]:
# Prerequisites
import utils
import itertools
import numpy as np
from copy import copy, deepcopy
from heapq import heappop, heappush

from utils import ZILE as DAYS
from utils import INTERVALE as INTERVALS
from utils import SALI as CLASSROOMS
from utils import MATERII as SUBJECTS
from utils import PROFESORI as TEACHERS

CONSTRAINTS = 'Constrangeri'
PREFERRED = 'Preferred'
NOT_PREFERRED = 'Not_preferred'
CAPACITY = 'Capacitate'
MAX_INTERVALS = 7

A* approach

In [224]:
def init_empty_timetable(data):
    '''Returns an empty timetable.'''

    timetable = {day: {interval: {classroom: None for classroom in data[CLASSROOMS]} for interval in data[INTERVALS]} for day in data[DAYS]}
    return timetable


def is_covered(state, subject, data):
    '''Checks the timetable to see if the subject is fully covered.'''

    student_count = 0


    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][1] == subject:
                    student_count += data[CLASSROOMS][classroom][CAPACITY]

    return student_count >= data[SUBJECTS][subject]


def count_teacher_slots(state, teacher):
    '''Counts the number of slots assigned to a teacher in the timetable.'''

    count = 0

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][0] == teacher:
                    count += 1

    return count

def choose_teacher(state, subject, data):
    # Choose the teacher that has the least amount of classed for this subject assigned to him.

    teacher_counts = {teacher: 0 for teacher in data[TEACHERS] if subject in data[TEACHERS][teacher][SUBJECTS]}

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][1] == subject:
                    teacher_counts[state[day][interval][classroom][0]] += 1

    # Filter out teachers that have already reached the maximum number of classes.
    teacher_counts = {teacher: count for teacher, count in teacher_counts.items() if count_teacher_slots(state, teacher) < MAX_INTERVALS}

    return min(teacher_counts, key=teacher_counts.get)


def check_hard_constraints(state, data):
    hard_cost = 0

    # Check if a teacher is assigned to more than one class at the same time.
    for day in state:
        for interval in state[day]:
            teachers = [state[day][interval][classroom][0] for classroom in state[day][interval] if state[day][interval][classroom] is not None]

            if len(teachers) != len(set(teachers)):
                hard_cost += 1

    # Check if a subject cannot be taught in the assigned classroom.
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue
                
                subject = state[day][interval][classroom][1]

                if subject not in data[CLASSROOMS][classroom][SUBJECTS]:
                    hard_cost += 1
    
    return hard_cost
        
def heuristic(state, data):
    '''Heuristic function.'''
    return 0

def is_final(state, data):
    '''The state is final if all subjects are covered, no hard constraints are violated and
    soft constraints are minimized.'''




def generate_next_states(state, data):
    '''Generate the neighbors of the current state.'''
    
    '''Identify all empty slots in the timetable where a subject can be assigned.
        For each empty slot, try assigning all possible combinations of teachers and subjects that satisfy the hard constraints.
        Check each assignment to ensure that it doesn't violate any of the hard constraints.
        If a valid assignment is found, create a new state representing the timetable with the assignment made.
        Repeat this process for all empty slots, generating a list of all possible neighbor states.'''
    
    next_states = []

    for subject in data[SUBJECTS]:

        if is_covered(state, subject, data):
            continue

        for day in state:
            for interval in state[day]:
                for classroom in state[day][interval]:

                    if state[day][interval][classroom] is None:
                        new_state = deepcopy(state)

                        teacher = choose_teacher(state, subject, data)
                        
                        new_state[day][interval][classroom] = (teacher, subject)

                        if check_hard_constraints(new_state, data) == 0:
                            next_states.append(new_state)


    # Check if the timetable covers all subjects.
    for subject in data[SUBJECTS]:
        if not is_covered(state, subject, data):
            return next_states                      

    return next_states


    

def astar(initial_state, heuristic, generate_next_states, is_final, data):
    '''A* algorithm.'''
    
    state = initial_state

    while not all(is_covered(state, subject, data) for subject in data[SUBJECTS]):
        next_states = generate_next_states(state, data)
        next_states.sort(key=lambda x: heuristic(x, data))

        if not next_states:
            break

        state = next_states[0]  
    
    return state


In [225]:
def run_astar(input_file):
    # Read input data.
    data = utils.read_yaml_file(input_file)

    # Sort subjects by number of students.
    data[SUBJECTS] = dict(sorted(data[SUBJECTS].items(), key=lambda item: item[1]))

    preferred = {}
    not_preferred = {}

    # Process the soft constraints for each teacher.
    for teacher in data[TEACHERS]:
        preferred = [constraint for constraint in data[TEACHERS][teacher][CONSTRAINTS] if not constraint.startswith('!')]
        not_preferred = [constraint.strip('!') for constraint in data[TEACHERS][teacher][CONSTRAINTS] if constraint.startswith('!')]


        # Convert intervals from '8-12' to (8, 10) and (10, 12).
        preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in preferred if interval[0].isdigit()]
        not_preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in not_preferred if interval[0].isdigit()]

        # Split the intervals in subintervals of 2 hours.
        preferred_intervals = [interval for start, end in preferred_intervals for interval in [(start, start + 2), (start + 2, end)]]
        not_preferred_intervals = [interval for start, end in not_preferred_intervals for interval in [(start, start + 2), (start + 2, end)]]
        
        # Remove intervals that have same start and end time.
        preferred_intervals = [str(interval) for interval in preferred_intervals if interval[0] != interval[1]]
        not_preferred_intervals = [str(interval) for interval in not_preferred_intervals if interval[0] != interval[1]]


        # Create the preferred and not preferred days.
        preferred_days = [day for day in preferred if not day[0].isdigit()]
        not_preferred_days = [day.strip('!') for day in not_preferred if not day[0].isdigit()]
        
        data[TEACHERS][teacher][PREFERRED] = preferred_days + preferred_intervals
        data[TEACHERS][teacher][NOT_PREFERRED] = not_preferred_days + not_preferred_intervals

    
    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

    # Run the A* algorithm.
    result = astar(initial_state, heuristic, generate_next_states, is_final, data)
    return result

In [226]:
input_files = ['dummy', 'orar_mic_exact', 'orar_mediu_relaxat', 'orar_mare_relaxat', 'orar_constrans_incalcat', 'orar_bonus_exact']

def transform_intervals(timetable):
    '''Function that transforms the intervals of the timetable from strings to tuples,
    in order to work with the utils.pretty_print function.'''

    new_timetable = {}

    for day in timetable:
        new_timetable[day] = {}

        for interval in timetable[day]:
            temp = interval.strip('()').split(',')
            new_interval = (int(temp[0]), int(temp[1]))
            new_timetable[day][new_interval] = timetable[day][interval]

    return new_timetable

def test_astar():
    for input_file in input_files:
        result = run_astar('inputs/' + input_file + '.yaml')

        with open('outputs/' + input_file + '.txt', 'w') as f:
           f.write(utils.pretty_print_timetable(transform_intervals(result), 'inputs/' + input_file + '.yaml'))

test_astar()                        